In [1]:
from flask import Flask, jsonify, request
import json
import numpy as np
import pickle
import pandas as pd
from car_model_building import Encoder


with open("car_model.pkl", "rb") as f:
    model = pickle.load(f)

if __name__=='__main__':
    with open('encoder.pkl', 'rb') as t:
        p_encoder = pickle.load(t)
        
        
app = Flask(__name__)


@app.route('/')
def index():
    return \
    """
    <!DOCTYPE html>
    <html>
    <body>
    
    <h2>Predict Prices of Used Cars in San Francisco Bay Area</h2>
    
    <form method="POST" action="/result">
    
      Car Model (Year):<br>
      <input type="number" name="year" value=50>
      <br><br>
      
      Car Model (Maker):<br>
      <input type="text" name="maker" value=60>
      <br><br>
      
      Car Condition:<br>
      <input type="radio" name="condition" value="salvage"> Salvage<br>
      <input type="radio" name="condition" value="fair"> Fair<br>
      <input type="radio" name="condition" value="good"> Good<br>
      <input type="radio" name="condition" value="like_new"> Like New<br>
      <input type="radio" name="condition" value="new"> New<br>
      <input type="radio" name="condition" value="excellent"> Excellent
      <br><br>
      
      Number of Cylinders:<br>
      <input type="number" name="cylinder" value=50>
      <br><br>
      
      Drive:<br>
      <input type="radio" name="drive" value="fwd"> FWD<br>
      <input type="radio" name="drive" value="rwd"> RWD<br>
      <input type="radio" name="drive" value="four_wd"> 4WD
      <br><br>
      
      Fuel:<br>
      <input type="radio" name="fuel" value="gas"> Gas<br>
      <input type="radio" name="fuel" value="diesel"> Diesel<br>
      <input type="radio" name="fuel" value="hybrid"> Hybrid<br>
      <input type="radio" name="fuel" value="electric"> Electric<br>
      <input type="radio" name="fuel" value="other"> Other
      <br><br>
      
      Odometer:<br>
      <input type="number" name="odometer" value=50>
      <br><br>
      
      Color:<br>
      <select name='color'>
          <option value="silver">silver</option>
          <option value="red">red</option>
          <option value="black">black</option>
          <option value="brown">brown</option>
          <option value="brown">brown</option>
          <option value="grey">grey</option>
          <option value="white">white</option>
          <option value="blue">blue</option>
          <option value="green">green</option>
          <option value="orange">orange</option>
          <option value="yellow">yellow</option>
          <option value="purple">purple</option>
      </select>
      <br><br>

      
      Car Size:<br>
      <input type="radio" name="size" value="full_size"> Full Size<br>
      <input type="radio" name="size" value="mid_size"> Middle Size<br>
      <input type="radio" name="size" value="compact"> Compact<br>
      <input type="radio" name="size" value="sub_compact"> Sub Compact
      <br><br>
      
      
      Transmission:<br>
      <input type="radio" name="trans" value="automatic"> Automatic<br>
      <input type="radio" name="trans" value="manual"> Manual<br>
      <input type="radio" name="trans" value="other"> Other
      <br><br>
      
      
      Car Type:<br>
      <select name='type'>
          <option value="truck">truck</option>
          <option value="sedan">sedan</option>
          <option value="suv">suv</option>
          <option value="convertible">convertible</option>
          <option value="hatchback">hatchback</option>
          <option value="coupe">coupe</option>
          <option value="wagon">wagon</option>
          <option value="offroad">offroad</option>
          <option value="mini_van">mini van</option>
          <option value="bus">bus</option>
          <option value="other">other</option>
      </select>
      <br><br>
      
      <input type="submit" value="Submit">
    </form> 
    </body>
    </html>
    """


@app.route('/result', methods=['POST'])
def result():
    year = request.form["year"]
    maker = request.form["maker"]
    condition = request.form["condition"]
    cylinder = request.form['cylinder']
    drive = request.form['drive']
    odometer = request.form['odometer']
    color = request.form['color']
    size = request.form['size']
    transmission = request.form['trans']
    types = request.form['type']
    
    features = ['price', 'year', 'maker', 'condition', 'cylinders', 'drive', 'fuel',
       'odometer', 'paint color', 'size', 'transmission', 'type']
    tmp = pd.DataFrame([year, maker, condition, cylinder, drive, odometer, color, size, transmission, types], 
                      colums = features)
    
    X = p_encoder.transform(tmp)
    
    pred = model.predict(X)
    return \
    """
    <!DOCTYPE html>
    <html>
    <body>
    
    The Estimated Price for this Car is: <br><br>
    {0}<br><br>
    
    <form action="/">
      <input type="submit" value="Recalculate">
    </form> 
    </body>
    </html>
    """.format(pred)


@app.route('/scoring', methods=['POST'])
def get_keywords():

    year = request.json["year"]
    maker = request.json["maker"]
    condition = request.json["condition"]
    cylinder = request.json['cylinder']
    drive = request.json['drive']
    odometer = request.json['odometer']
    color = request.json['color']
    size = request.json['size']
    transmission = request.json['trans']
    types = request.json['type']
    
    features = ['price', 'year', 'maker', 'condition', 'cylinders', 'drive', 'fuel',
       'odometer', 'paint color', 'size', 'transmission', 'type']
    tmp = pd.DataFrame([year, maker, condition, cylinder, drive, odometer, color, size, transmission, types], 
                      colums = features)
    X = p_encoder.transform(tmp)
    results = {"car_price":model.predict(X)}
    return jsonify(results)


if __name__ == "__main__":
    app.run(debug=True, host='127.0.0.1', port=5000)

/Users/hercule/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/hercule/anaconda3/lib/python3.6/site-packages/sklearn/impute/_iterative.py:638: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with fsevents reloader


SystemExit: 1

/Users/hercule/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
